In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/googlenet-image/Screenshot 2024-09-21 103339.png


In [6]:
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [8]:
img_path = '/kaggle/input/googlenet-image/Screenshot 2024-09-21 103339.png'
img = Image.open(img_path)
plt.figure(figsize=(img.size[0]/10, img.size[1]/10)) 
plt.imshow(img)
plt.axis('off') 
plt.show()

## --------------------------code here-------------------

### Note that
#### I would not be testing the same on a dataset,this is just the code of GoogleNet

# Imports

In [9]:
import torch
import torch.nn as nn

In [18]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, **kwargs) # kwargs for varying kernel size and padding
        self.batchnorm = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [27]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super().__init__()
        self.branch1 = conv_block(in_channels, out_1x1, kernel_size = 1)
        
        self.branch2 = nn.Sequential(
            conv_block(in_channels, red_3x3,kernel_size = 1),
            conv_block(red_3x3, out_3x3, kernel_size = 3, padding = 1))
        
        self.branch3 = nn.Sequential(
            conv_block(in_channels, red_5x5,kernel_size = 1),
            conv_block(red_5x5, out_5x5, kernel_size = 5, padding = 2))
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size = (3,3), stride = 1, padding = 1),
            conv_block(in_channels, out_1x1pool, kernel_size = 1))
        
    def forward(self, x):
        # input dim = batch_size x in_channels x size x size  ===> thus,, concatenate will take place at index 1 
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

In [30]:
class GoogleNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 1000):
        super().__init__()
        self.conv1 = conv_block(in_channels = in_channels, out_channels = 64, kernel_size = 7, stride = 2, padding = 3)
        self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = conv_block(64, 192, kernel_size = 3, stride =1, padding = 1)
        self.pool2 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        #  in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool
        self.inception3a = InceptionBlock(192,64 ,96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256,128, 128, 192, 32, 96, 64)
        self.pool3 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        self.inception4a = InceptionBlock(480,192 ,96, 208, 16, 48 ,64)
        self.inception4b = InceptionBlock(512,160, 112, 224, 24 ,64, 64)
        self.inception4c = InceptionBlock(512,128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512,112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528,256, 160, 320, 32, 128 ,128)
        self.pool4 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832,384, 192, 384, 48 ,128, 128 )
        self.avgpool = nn.AvgPool2d(kernel_size = 7, stride = 1)
        self.drop = nn.Dropout(0.4)
        self.linear = nn.Linear(1024,num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.pool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.pool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.drop(x)
        x = self.linear(x)
        
        return x

## check

In [31]:
sample = torch.randn(5, 3, 224, 224)  # images of size 3x224x224
model = GoogleNet()
print(model(sample).shape)

torch.Size([5, 1000])


# hurrrayyyyyy !!!!